#Installs

In [ ]:
!pip install streamlit langchain OpenAI -q

In [ ]:
# Bring in deps
import os 
from apikey import apikey 
import datetime
import json

from langchain.llms import OpenAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import PALChain
from langchain.agents import initialize_agent, Tool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from typing import Optional, Type      
from langchain.tools import BaseTool, tool
os.environ['OPENAI_API_KEY'] = apikey

#Tool functions

In [ ]:
# Define the reworked functions

def add_event_from_string(input_string):
    name, datetime_input = input_string.split(",", 1)
    name = name.strip()
    datetime_input = datetime_input.strip()

    event_datetime = datetime.datetime.strptime(datetime_input, "%Y-%m-%d-%H-%M")
    date_str = event_datetime.strftime("%Y-%m-%d")
    time_str = event_datetime.strftime("%H:%M")
    event = {"name": name, "date_and_time": f"{date_str} {time_str}"}

    if date_str not in calendar:
        calendar[date_str] = {}

    if time_str not in calendar[date_str]:
        calendar[date_str][time_str] = []

    calendar[date_str][time_str].append(event)

def from_json(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            calendar = json.load(f)
    else:
        calendar = {}
    return calendar

def save_calendar_as_json(filename):
    with open(filename, 'w') as file:
        json.dump(calendar, file)
    #print(f"Calendar saved as {filename}.")

def get_date(ignore_input=None):
    now = datetime.datetime.now()
    year = now.year
    month = now.month
    day = now.day
    return year, month, day

def extract_dates(ignore_input=None):
    result = []
    calendar = {
        '2023-06-01': {
            '12:00': [
                {
                    'name': 'Doctors Appointment',
                    'date_and_time': '2023-06-01 12:00'
                }
            ]
        }
    }
    
    for date, time_slots in calendar.items():
        for time, appointments in time_slots.items():
            for appointment in appointments:
                name = appointment['name']
                date_and_time = appointment['date_and_time']
                result.append((name, date_and_time))
    return result

#Tool and memory setup

In [ ]:
llm = OpenAI(temperature=0)
pal_chain = PALChain.from_math_prompt(llm, verbose=True)
tools = [
	Tool(
		name = "Current date",
		func =  get_date,
		description= "Useful for when you need to check what it currently is. There is no input required, and it will output a tuple with 3 variables; year, month, day. You should simply call the function: get_date()",
		),
	Tool(
		name = "Check calendar",
		func = extract_dates,
		description = "Useful for when you need to check all planned events in the calendar. There is no input required, and it will output a list of all planned events. You should simply call the function: extract_dates()"	
		),
	Tool(
		name = 'add event from string',
		func = add_event_from_string,
		description="Useful for when you need to schedule an event for a given date and time. The input should be as the following example: 'name_of_event, 2023-05-21-12-30'",
		),
	Tool(
		name = 'save calendar as json',
		func = save_calendar_as_json,
		description = "Useful for when you need to save calendar information as a json file. Example of usage: save_calendar_as_json('events.json')"
		),
	Tool(
		name = "from json",
		func = from_json,
		description = "Useful for when you need to load saved events from json file. This tool should always be used like this example: calendar = from_json('events.json')"	
	  ),
	Tool(
        name = "PAL",
        func = pal_chain.run,
        description = "useful for when you need to answer questions about math or word problems or date calculations/comparisons"),
]

memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
agent_chain = initialize_agent(
		tools=tools,
		llm=llm,
		agent='conversational-react-description', 
		memory=memory, 
		verbose=True
		)

#Prompt engineering

In [ ]:
sys_message = """ Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

TOOLS:
------

Assistant has access to the following tools:

> Current date: useful to get today's date. There is no input required, and it will output 3 variables; year, month, day. You should simply call the function: get_date()
> check events: Use to check on what events are coming up and needs to taken into account. The input to this tool should be a string in this format year, month, day. This is the only way for you to answer questions about upcoming events. This tool will reply with a list of planned events for the specified date in 24hour time.
> add event from string: Useful to schedule an event for a given date and time, and add it to calendar dictionary. The input should be as the following example: 'name_of_event, 2023-05-21-12-30'
> save calendar as json: Useful for storing calendar information as a json file. You always have to store it as events.json. example of usage: save_calendar_as_json('events.json')
> from json: Use to load saved events from json file. The json file should always be named 'events.json'
> PAL: useful for when you need to answer questions about math or word problems or date calculations/comparisons

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [Current date, check events, add event from string, save calendar as json, from json, PAL]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
AI: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad} """

In [ ]:
agent_chain.agent.llm_chain.prompt.template = sys_message

#Running the agent model

In [ ]:
calendar = {} # We clear out the events dictionary for testing purposes

In [ ]:
#AgentCall function
def ScheduleAgent(input_string):
  calendar = from_json('events.json')
  agent_chain.run('What is todays date?')
  agent_chain.run(f'{input_string}')
  save_calendar_as_json('events.json')

In [ ]:
AgentCall("I want to schedule a doctors appointment for 1st of june at 12:00.")

#Current issues:

- General inconsistency: Sometimes the model doesn't correctly save an event to dictionary, causing the calendar variable as well as 'events.json' to be output as empty.
- add_event_from_string functions does work properly, however it has unnecessary code smell in terms of the try: except:. The exception error seems to always be thrown regardless of function input.